Subword Tokenizer에 대해 알아보기
자연어처리 모델을 훈련시킬 때, tokenizing 된 단어의 개수는 모델 성능에 다양한 영향을 미친다.
기계가 모르는 단어로 인해 문제를 푸는 것이 까다로워질 수 있기 때문이다.

이러한 상황을 해결하기 위해 나온 것이 Subword Segmentation이다.
하나의 단어를 여러 subword로 분리해서 단어를 인코딩/임베딩하겠다는 의도이다.

BPE (Byte Pair Encoding)
연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식

ex.
#dictionary
low: 5, lower : 2, newest : 6, widest : 3
vocabulary l, o, w, e, r, n, w, s, t, i, d

1회 - 빈도수가 9로 가장 높은 (e,s) 쌍을 es로 통합
low: 5, lower : 2, new(es)t: 6, wid(es)t : 3
vocabulary l, o, w, e, r, n, w, s, t, i, d, es

2회 - 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합
low: 5, lower : 2, new(est): 6, wid(est) : 3
vocabulary l, o, w, e, r, n, w, s, t, i, d, es, est

3회 - 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합
(lo)w : 5, (lo)wer : 2, newest : 6, widest : 3
...
10회
vocabulary update! l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

lowest를 low와 est 두 단어로 인코딩


Wordpiece Model (WPM)
- BPE의 변형 알고리즘
- 병합되었을 때 코퍼스의 Likelihood를 가장 높이는 쌍을 병합
    - 띄어쓰기: subword를 구분하는 구분자
    - _(언더바): 실제 문장의 띄어쓰기

WPM을 수행하기 이전의 문장: Jet makes feud over seat width with big orders at stake
WPM을 수행한 결과(wordpieces): _J et _makers _fe ud _over _seat _width _with _big _orders _at _stake

출처: https://data-make.tistory.com/692

SentencePiece
- 빈도수를 기반으로 BPE를 수행
1. pre-tokenization을 필요로 하지 않는다.
2. character-coverage 설정이 가능하다. (빈도에 따라 상위99%의 character들만 고려하는 등의 옵션을 주는 것이 가능)
3. subword generalization이 가능하다. (다른 token들도 샘플링하면서 사용하여 overfitting하는 것이 가능)
4. 속도가 빠르다.

출처: https://process-mining.tistory.com/191

In [1]:
!!pip install transformers

['Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/',
 'Collecting transformers',
 '  Downloading transformers-4.27.3-py3-none-any.whl (6.8 MB)',
 '\x1b[?25l     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b0.0/6.8 MB\x1b \x1b?\x1b eta \x1b-:--:--\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━\x1b\x1b╸\x1b\x1b━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b2.7/6.8 MB\x1b \x1b79.8 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b6.8/6.8 MB\x1b \x1b99.8 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b\x1b╸\x1b \x1b6.8/6.8 MB\x1b \x1b99.8 MB/s\x1b eta \x1b0:00:01\x1b',
 '\x1b[2K     \x1b━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━\x1b \x1b6.8/6.8 MB\x1b \x1b51.9 MB/s\x1b eta \x1b0:00:00\x1b',
 '\x1b[?25hRequirement already satisfied: filelock in /usr/local/lib/python3.9/dist-packages (from transformers) (3.10.2)',
 'Requirement already satisfied: requests in 

In [2]:
from transformers import pipeline

pipe = pipeline("text-classification")
pipe(['Daewoo Hall is too far from the main gate'])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


[{'label': 'NEGATIVE', 'score': 0.9895686507225037}]

In [5]:
from transformers import pipeline
classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
classifier('Daewoo Hall is too far from the main gate')

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'anger', 'score': 0.015530571341514587},
  {'label': 'disgust', 'score': 0.02225101739168167},
  {'label': 'fear', 'score': 0.021104635670781136},
  {'label': 'joy', 'score': 0.007998050190508366},
  {'label': 'neutral', 'score': 0.6047179698944092},
  {'label': 'sadness', 'score': 0.050953157246112823},
  {'label': 'surprise', 'score': 0.27744463086128235}]]